In [1]:
import torch
print(torch.__version__)
print(f"MPS available: {torch.backends.mps.is_available()}")

2.6.0
MPS available: True


In [1]:
import os

from peft import AutoPeftModelForCausalLM
import transformers
from transformers import AutoTokenizer
import torch


path = "/Users/teasgen/workspace/diploma/llm_backend/lora_model_mark_answer_comment_first_system_prompt"

model = AutoPeftModelForCausalLM.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
).to("mps")
tokenizer = AutoTokenizer.from_pretrained(path)

/Users/teasgen/miniconda3/envs/django_tests/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  9.54it/s]


In [2]:
system = system = """Ты проводишь онлайн собеседование. Прокомментируй ответ пользователя. Дай ему оценку от 1 до 10:
1 - пользователь ответил абсолютно нерелевантную информацию
2 - пользователь ответил нерелевантную информацию, однако упомянул верный факт про машинное обучение
3 - пользователь ответил нерелевантную информацию, однако упомянул верный факт, затрагивающий тему
4 - пользователь ответил на вопрос очень кратно, объяснил поверхностно
5 - пользователь ответил на вопрос кратко, однако указал верные факты, допускаются неточности
6 - пользователь ответил на вопрос, указал верные факты, допускаются неточности. Однако ответ требует дополнений, могут быть упущены какие-то важные аспекты
7 - пользователь ответил на вопрос, указал верные факты, допускаются неточности. Однако ответ требует дополнений, все важные аспекты должны быть упомянуты
8 - пользователь ответил на вопрос, указал верные факты, неточностей быть не должно. Однако ответ требует дополнений - примеров, все важные аспекты должны быть упомянуты
9 - пользователь ответил на вопрос, указал верные факты, неточностей быть не должно. Ответ точен и полон, все важные аспекты должны быть упомянуты, ответ расширает вопрос, а не отвечает напрямую
10 - пользователь ответил на вопрос полностью корректно, дополнений быть не может. Идеальный ответ.
При неполном ответе придумай вопрос с уточнением и напиши его. Если ответ на 10, то можешь придумать дополнительный вопрос. Ответ должен быть в json формате с полями 'Оценка', 'Новый вопрос', 'Комментарий проверяющей системы' в строго указанном порядке\n"""
prompt = f"{{\"Вопрос\": \"Что такое линейная регрессия?\", \"Ответ пользователя\": \"Линейная регрессия — это метод предсказания значений.\"\}}"

In [3]:
from transformers import TextIteratorStreamer, TextStreamer

messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": prompt}
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("mps")

text_streamer = TextStreamer(tokenizer, skip_prompt = True)

In [4]:
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1000,
    use_cache = True, temperature = 1.5, min_p = 0.1)

{"Оценка": "1.0", "Новый вопрос": "Какую функцию используется в линейной регрессии для измерения разницы между истинными и предсказанными значениями?", "Комментарий проверяющей системы": "Ответ пользователя крайне неправилен, 

KeyboardInterrupt: 

In [39]:
model.eval()

messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

KeyboardInterrupt: 

In [24]:
response

['"Комментарий проверяющей системы": "Ответ пользователя является слишком кратким и не раскрывает сути линейной регрессии. Линейная регрессия — это статистический метод для моделирования линейных зависимостей между переменными, использующийся для предсказания значения целевой переменной на основе одного или нескольких предикторов. Она основана на минимизации суммы квадратов отклонений между наблюдаемыми значениями и прогнозируемыми. Важным аспектом также является использование прямой линии для представления зависимости. Ответ мог бы быть улучшен, если бы он включал описание математической формулы, использованной в линейной регрессии, и пояснение её применимости.", "Оценка": "4.0"']

In [31]:
from threading import Thread

from transformers import TextIteratorStreamer

messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": prompt}
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("mps")

text_streamer = TextIteratorStreamer(tokenizer, skip_prompt = True)

generation_kwargs = dict(input_ids=inputs, streamer=text_streamer, max_new_tokens=512)
thread = Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()

generated_text = ""
for new_text in text_streamer:
    print(new_text)
    generated_text += new_text
# print(generated_text)




{"Новый 

вопрос": 


"Какова 



математическая 

основа 



линейной 



регрессии 
и 
как 
она 

применяется 
для 


построения 
модели 




предсказания?", 





"Комментарий 


проверяющей 

системы": 


"Ответ 
пользователя 



неверный, 
так 
как 
он 
слишком 
общ 
и 
не 


описывает 

сути 



линейной 




регрессии. 



Линейная 



регрессия 
— 
это 
метод 




KeyboardInterrupt: 

In [2]:
import requests

url = "http://127.0.0.1:8000/generate-question/"
data = {
    "prompt": "Придумай строго один теоретический вопрос по машинному обучению по теме Свёрточные сети",
    "use_validation_system_prompt": False,
}

response = requests.post(url, json=data, stream=True)

for chunk in response.iter_lines(chunk_size=1):
    if chunk:
        print(chunk.decode('utf-8'))

Какие 
архитектурные 
компоненты 
и 
концепции 
играют 
ключевую 
роль 
в 
свёрточных 
нейронных 
сетях 
(CNN) 
для 
обработки 
изображений, 
и 
как 
они 
способствуют 
эффективной 
выделению 
признаков?<|im_end|>
